<a href="https://colab.research.google.com/github/lia-ang/BME3053C_ECG_Project/blob/main/final_project_ecg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Group Project: Angela Liang, Ece Sarioglu, Ankitha Nath, Siri Yarrapatni, Vivian Lam, Alana Young

Repositories: https://github.com/huggingface/transformers/tree/main/src/transformers/models/longformer
https://github.com/allenai/longformer

Dataset: https://physionet.org/content/ecg-arrhythmia/1.0.0/

In [2]:
# Import libraries for data manipulation and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import libraries for signal processing
from scipy.signal import butter, lfilter, welch

# Import libraries for machine learning and evaluation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Import PyTorch and Hugging Face Transformers for Longformer
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import LongformerTokenizer, LongformerModel, LongformerForSequenceClassification

# Import additional utilities
import os
import warnings
warnings.filterwarnings('ignore')

# For reproducibility
np.random.seed(42)
torch.manual_seed(42)

/home/codespace/.python/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Install required libraries
#pip install wfdb
#pip install transformers
#pip install datasets
#pip install scikit-learn
# pip install huggingface_hub[hf_xet]


In [4]:
# Import necessary libraries
import os
import wfdb
import matplotlib.pyplot as plt

# Define the folder containing WFDB records
folder_path = "WFDBRecords"

# Iterate through all files in the folder
for file in os.listdir(folder_path):
    if file.endswith(".hea"):  # Process only header files
        record_name = os.path.splitext(file)[0]  # Get the record name without extension
        record_path = os.path.join(folder_path, record_name)
        
        # Load the record
        record = wfdb.rdrecord(record_path)
        signal = record.p_signal
        
        # Print the shape of the signal
        print(f"Processing {record_name}: Shape = {signal.shape}")
        
        # Plot the first lead (e.g., Lead I)
        plt.figure()
        plt.plot(signal[:, 0])
        plt.title(f"Lead I - ECG Signal from {record_name}")
        plt.xlabel("Time (samples)")
        plt.ylabel("Amplitude (mV)")
        plt.grid(True)
        plt.show()

In [5]:
import os
import wfdb
import numpy as np
from sklearn.model_selection import train_test_split

# Define the folder containing WFDB records
folder_path = "WFDBRecords"

# Initialize lists to store signals and labels
signals = []
labels = []

# Recursively search for .hea files in all subdirectories
for root, _, files in os.walk(folder_path):
    for file in files:
        if file.endswith(".hea"):  # Process only header files
            record_name = os.path.splitext(file)[0]  # Get the record name without extension
            record_path = os.path.join(root, record_name)
            
            # Load the record
            try:
                record = wfdb.rdrecord(record_path)
                signal = record.p_signal
                
                # Append the first lead (e.g., Lead I) to the signals list
                signals.append(signal[:, 0])  # Assuming Lead I is the first column
                
                # Append a placeholder label (e.g., 0 or 1) to the labels list
                labels.append(0)  # Replace with actual label logic
                
                # Free memory by processing in smaller chunks
                if len(signals) >= 100:  # Process every 100 signals
                    signals = np.array(signals, dtype=object)
                    labels = np.array(labels)
                    
                    # Save or process the batch here (e.g., write to disk or train model)
                    print(f"Processed batch of 100 signals. Total signals processed: {len(signals)}")
                    
                    # Clear memory
                    signals = []
                    labels = []
            except Exception as e:
                print(f"Error processing {record_name}: {e}")

#convert signals to float32 for compatibility with PyTorch
signals = [np.array(signal, dtype=np.float32) for signal in signals]

# Final processing for remaining signals
if len(signals) > 0:
    signals = np.array(signals, dtype=object)
    labels = np.array(labels)
    print(f"Processed final batch of {len(signals)} signals.")

Processed batch of 100 signals. Total signals processed: 100
Processed batch of 100 signals. Total signals processed: 100
Processed batch of 100 signals. Total signals processed: 100
Processed batch of 100 signals. Total signals processed: 100
Processed batch of 100 signals. Total signals processed: 100
Processed batch of 100 signals. Total signals processed: 100
Processed batch of 100 signals. Total signals processed: 100
Processed batch of 100 signals. Total signals processed: 100
Processed batch of 100 signals. Total signals processed: 100
Processed batch of 100 signals. Total signals processed: 100
Processed batch of 100 signals. Total signals processed: 100
Processed batch of 100 signals. Total signals processed: 100
Processed batch of 100 signals. Total signals processed: 100
Processed batch of 100 signals. Total signals processed: 100
Processed batch of 100 signals. Total signals processed: 100
Processed batch of 100 signals. Total signals processed: 100
Processed batch of 100 s

In [6]:
import os
import wfdb
import numpy as np
from scipy.signal import butter, lfilter
from sklearn.model_selection import train_test_split
from transformers import LongformerTokenizer

# Define a bandpass filter
def bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype="band")
    return lfilter(b, a, data)

# Sampling frequency and filter parameters
fs = 360
lowcut = 0.5
highcut = 50

# Define the folder containing WFDB records
folder_path = "WFDBRecords"

# Initialize tokenizer
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")
max_length = 2048  # Reduce max length

# Initialize lists to store signals and labels
signals = []
labels = []

# Process all .hea files in the folder
file_limit = 100  # Process only the first 100 files
file_count = 0

for root, _, files in os.walk(folder_path):
    for file in files:
        if file.endswith(".hea"):
            file_count += 1
            if file_count > file_limit:
                break
            
            record_name = os.path.splitext(file)[0]
            record_path = os.path.join(root, record_name)
            try:
                # Load the record and filter the signal
                record = wfdb.rdrecord(record_path)
                signal = record.p_signal[:, 0]  # Use the first lead
                filtered_signal = bandpass_filter(signal, lowcut, highcut, fs)
                
                # Tokenize the signal
                tokenized_signal = tokenizer(
                    " ".join(map(str, filtered_signal)),
                    truncation=True,
                    padding="max_length",
                    max_length=4096,
                    return_tensors="pt"
                )
                
                # Append the tokenized signal and a placeholder label
                signals.append(tokenized_signal["input_ids"].squeeze(0).tolist())
                labels.append(0)  # Replace with actual labels if available
            except Exception as e:
                print(f"Error processing {record_name}: {e}")

# Convert to numpy arrays
signals = np.array(signals, dtype=object)
labels = np.array(labels)

# Split data into training, validation, and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(signals, labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)

print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")
print(f"Testing set size: {len(X_test)}")

Training set size: 60
Validation set size: 20
Testing set size: 20


In [7]:
# Custom Dataset Class
import torch
from torch.utils.data import Dataset

class ECGDataset(Dataset):
    def __init__(self, signals, labels, tokenizer, max_length=4096):
        self.signals = signals
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.signals)

    def __getitem__(self, idx):
        # Tokenize the signal
        tokenized = self.tokenizer(
            " ".join(map(str, self.signals[idx])),
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )
        return {
            "input_ids": tokenized["input_ids"].squeeze(0),
            "attention_mask": tokenized["attention_mask"].squeeze(0),
            "label": torch.tensor(self.labels[idx], dtype=torch.long)
        }

# Create datasets
# Create datasets
train_dataset = ECGDataset(X_train, y_train, tokenizer=tokenizer)
val_dataset = ECGDataset(X_val, y_val, tokenizer=tokenizer)
test_dataset = ECGDataset(X_test, y_test, tokenizer=tokenizer)

In [ ]:
# Define the Longformer Model

from transformers import LongformerForSequenceClassification

# Load Longformer model
model = LongformerForSequenceClassification.from_pretrained("allenai/longformer-base-4096", num_labels=2)  # Adjust num_labels for your task

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


: 

In [ ]:
# Training the Model

from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)  # Reduce batch size
val_loader = DataLoader(val_dataset, batch_size=2)
test_loader = DataLoader(test_dataset, batch_size=2)
# Optimizer and loss function
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = CrossEntropyLoss()

# Training loop with validation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(3):  # Number of epochs
    # Training phase
    model.train()
    total_train_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    
    # Validation phase
    model.eval()
    total_val_loss = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_val_loss += loss.item()
            preds = torch.argmax(outputs.logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # Print training and validation loss
    print(f"Epoch {epoch + 1}")
    print(f"Training Loss: {total_train_loss / len(train_loader)}")
    print(f"Validation Loss: {total_val_loss / len(val_loader)}")
    print(f"Validation Metrics:\n{classification_report(all_labels, all_preds)}")

Initializing global attention on CLS token...


In [ ]:
# Evaluate the Model

from sklearn.metrics import classification_report

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print(classification_report(all_labels, all_preds))